In [1]:
using Distributed
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
@everywhere using SharedArrays

In [3]:
function multiply_matrices(matr1, matr2, matr3)
    dim = size(matr1, 2)
    
    @sync @distributed for i in 1:dim
        for j in 1:dim
            temp = 0
            for k in 1:dim
                temp += matr1[i, k] * matr2[k, j]
            end
            matr3[i, j] = temp
        end
    end
end

multiply_matrices (generic function with 1 method)

In [17]:
function multiply_matrices_non_parallel(matr1, matr2, matr3)
    dim = size(matr1, 2)
    
    for i in 1:dim+1
        for j in 1:dim+1
            temp = 0
            for k in 1:dim+1
                temp += matr1[i, k] * matr2[k, j]
            end
            matr3[i, j] = temp
        end
    end
end

multiply_matrices_non_parallel (generic function with 1 method)

In [5]:
RESULTS = Dict(100 => Dict("par" => 0.0, "non_par" => 0.0, "iters" => 1000), 
    200 => Dict("par" => 0.0, "non_par" => 0.0, "iters" => 1000), 500 => Dict("par" => 0.0, "non_par" => 0.0, "iters" => 150), 
    750 => Dict("par" => 0.0, "non_par" => 0.0, "iters" => 50))

Dict{Int64,Dict{String,Real}} with 4 entries:
  200 => Dict{String,Real}("par"=>0.0,"non_par"=>0.0,"iters"=>1000)
  100 => Dict{String,Real}("par"=>0.0,"non_par"=>0.0,"iters"=>1000)
  750 => Dict{String,Real}("par"=>0.0,"non_par"=>0.0,"iters"=>50)
  500 => Dict{String,Real}("par"=>0.0,"non_par"=>0.0,"iters"=>150)

In [6]:
for (k, v) in RESULTS
    mt1 = rand(-100:100, k, k)
    mt2 = rand(-100:100, k, k)
    
    let i = 0
        while i < v["iters"]
            mt3 = convert(SharedArray, zeros(Int64, k, k))
            mt4 = convert(SharedArray, zeros(Int64, k, k))
            
            v["par"] += @elapsed multiply_matrices(mt1, mt2, mt3)
            v["non_par"] += @elapsed multiply_matrices_non_parallel(mt1, mt2, mt4)
            
            i += 1
        end
    end
    
    v["par"] /= v["iters"]
    v["non_par"] /= v["iters"]
    
    println("$k was done")    
end

200 was done
100 was done
750 was done
500 was done


In [7]:
RESULTS

Dict{Int64,Dict{String,Real}} with 4 entries:
  200 => Dict{String,Real}("par"=>4.6996e-5,"non_par"=>3.35217e-5,"iters"=>1000)
  100 => Dict{String,Real}("par"=>1.17015e-5,"non_par"=>3.65096e-6,"iters"=>100…
  750 => Dict{String,Real}("par"=>0.060991,"non_par"=>0.0941401,"iters"=>50)
  500 => Dict{String,Real}("par"=>0.00651459,"non_par"=>0.00913465,"iters"=>150)